# Vectorized K-Core decomposition

Import essential libraries from modules directory

In [1]:
import sys
sys.path.insert(0, './graphio')
sys.path.insert(0, './graphutils')

from graphio import GraphIO
from graphutils import GraphUtils

gio = GraphIO(verbose=0)
gul = GraphUtils(library="numpy")

Read dataset and Initialize required vectors

In [2]:
import numpy as np
import timeit
filename = "./datasets/sample_input_graph_02.txt"
src, dst, N = gio.read(filename)
_, II, D = np.unique(src, return_index=True, return_counts=True)
D = D.astype(np.uint32)
II = II.astype(np.uint32) # neighbors index
CD = D.copy() # copy of original D
C = N.copy()   #indices of nodes that are relevant
K = np.zeros_like(N)  # to store results

multi_arange = GraphUtils.multi_arange_numpy
unique = GraphUtils.unique_numpy_bin_count

%load_ext line_profiler

K-Core Decomposition Algorithm

In [3]:
def kcore_decomposition(I, II, C, D, CD, K, k=1):
    B = C[D[C]<=k]   #indices of nodes that will be deleted
    while C.shape[0] > 1:
        while B.shape[0] > 0:
            D[B] = 0
            K[B] = k

            #subtracting from neighbors
            J = I[multi_arange(II[B], CD[B])]
            H = J[D[J]>0]
            H, Cnt = unique(H)
            D[H] -= Cnt

            B = H[D[H] <= k]  #indices of nodes that will be deleted;

        k = k + 1
        C = C[D[C]>=k]   #indices of nodes that are relevant
        B = C[D[C]==k]   #indices of nodes that will be deleted
    return K

Run and profile the algorithm

In [4]:
#%lprun -f kcore_decomposition
%time K = kcore_decomposition(dst, II, C, D, CD, K)
print(f"Average K: {np.average(K)}")

CPU times: user 106 µs, sys: 3.09 ms, total: 3.2 ms
Wall time: 2.25 ms
Average K: 2.625


Validate the result with expected output.

In [5]:
import re
filename = re.sub("sample_input", "expected_output_kcore", filename)
arr_txt = np.loadtxt(filename, dtype=gio.dtype, comments="#")
print(f"Output Match: {np.array_equal(K, arr_txt[:, 1])}")

Output Match: True
